<a href="https://colab.research.google.com/github/hussainqadiim/-/blob/main/Used_Cars_Prices_in_UK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'used-cars-prices-in-uk:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3891476%2F6761207%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240214%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240214T140326Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3aafe4b4e776d1b740191e0f51a762b367ab07033e724e609fbf33beb99fd61a29d9f5dae02b317e39e7ef8080a256c36171448f5be6129ad79bd2fa5fd9e8b8b04118e6442499e11741651e6e6868690fb58623ebd4a12a5341b448a6c93b725a25a52bf274a282c4d7184733acc0de54192d7e4b98f0ce5d9f3159803d2e5c2cb126a6d278da3f2e810ec638402b651bc93063268badeb2f55f195dd3ec66b2c7c22280077881be2c6ff1a303f8141bd44a4f7260827fe4b9f8de88edc40591a34de724900a6c5419822dfb8151007581715f440ae8de892cc5fe6b5b3d17cfab683e3b187273e50dd8e8a9fba1662553a697de3cc6b3f8b202b5134700b01'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

pd.set_option("display.max_columns",None)

In [ ]:
TARGET = "Price"

# Loading Data

In [ ]:
df = pd.read_csv("/kaggle/input/used-cars-prices-in-uk/used_cars_UK.csv",index_col=0)
print(df.shape)
df.head(3)

# Data Observation

## all

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

## Price

In [ ]:
sns.histplot(df[TARGET])

## Mileage(miles)

In [ ]:
df = df[df["Mileage(miles)"]<1000000]

In [ ]:
sns.histplot(df["Mileage(miles)"])

In [ ]:
sns.scatterplot(x=df["Mileage(miles)"],y=df[TARGET])

# Registration_Year

In [ ]:
sns.histplot(df["Registration_Year"])

In [ ]:
df["Registration_Year"].value_counts()

In [ ]:
tmp_df = df.groupby("Registration_Year").agg({TARGET:"mean"})
sns.lineplot(x=tmp_df.index, y=tmp_df[TARGET])

## Previous Owners

In [ ]:
sns.histplot(df["Previous Owners"])

In [ ]:
sns.boxplot(x=df["Previous Owners"],y=df[TARGET])

In [ ]:
tmp_df = df.groupby("Previous Owners").agg({TARGET:"mean"})
sns.barplot(x=tmp_df.index,y=tmp_df[TARGET])

In [ ]:
sns.boxplot(x=df["Previous Owners"],y=df["Registration_Year"])

## Fuel type

In [ ]:
print(df["Fuel type"].value_counts())
plt.figure(figsize=(10,6))
sns.histplot(df["Fuel type"])

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x=df["Fuel type"],y=df[TARGET])

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x=df["Fuel type"],y=df["Registration_Year"])

## Body type

In [ ]:
print(df["Body type"].value_counts())
plt.figure(figsize=(10,6))
sns.histplot(df["Body type"])

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x=df["Body type"],y=df[TARGET])

In [ ]:
pd.crosstab(df["Body type"],df["Fuel type"],margins=True,normalize=True)

## Engine

In [ ]:
df["Engine"] = df["Engine"].str[:3].astype("float")

In [ ]:
sns.histplot(df["Engine"])

In [ ]:
sns.scatterplot(x=df["Engine"],y=df[TARGET])

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x=df["Fuel type"],y=df["Engine"])

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x=df["Body type"],y=df["Engine"])

## Gearbox

In [ ]:
df["Gearbox"].value_counts()

In [ ]:
sns.boxplot(x=df["Gearbox"],y=df[TARGET])

In [ ]:
sns.boxplot(x=df["Gearbox"],y=df["Registration_Year"])

In [ ]:
pd.crosstab(df["Fuel type"],df["Gearbox"],margins=True,normalize=True)

In [ ]:
pd.crosstab(df["Body type"],df["Gearbox"],margins=True,normalize=True)

## Doors

In [ ]:
sns.histplot(df["Doors"])

In [ ]:
sns.boxplot(x=df["Doors"],y=df[TARGET])

## Seats

In [ ]:
sns.histplot(df["Seats"])

In [ ]:
sns.boxplot(x=df["Seats"],y=df[TARGET])

In [ ]:
pd.crosstab(df["Doors"],df["Seats"],margins=True,normalize=True)

#### Odd doors are oddly constructed.

## Emission Class

In [ ]:
df["Emission Class"].value_counts()

In [ ]:
sns.boxplot(x=df["Emission Class"],y=df[TARGET])

In [ ]:
sns.boxplot(x=df["Emission Class"],y=df["Registration_Year"])

### 5.6 is exceptionally high. Recently manufactured ones tend to be 5.6
### What could be the reason why 1 is also high on average?

## Service history

In [ ]:
df["Service history"].fillna("None",inplace=True)

In [ ]:
sns.boxplot(x=df["Service history"],y=df[TARGET])